<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/llama_smishTank_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install llamaapi

In [4]:
import pandas as pd
import json
from llamaapi import LlamaAPI
from google.colab import userdata
from tqdm import tqdm
import re  # Ensure 're' module is imported
import time
from google.colab import drive  # Import the drive module from google.colab
import os


# Initialize the SDK
llama = LlamaAPI(userdata.get('LLAMA-API-KEY'))


In [5]:
# mount google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# this function does everything as the previous one except you can specify which model you want to use
def check_sms_legitimacy(message, model):
    request_message = f"Is this message real or fake. Just say real, fake, or undetermined: {message}"
    while True:
        try:
            response = llama.run({
                "model": model,
                "messages": [{"role": "user", "content": request_message}]
            })

            try:
                response_json = response.json()
                used_model = response_json.get('model', 'Unknown model')
                content = response_json['choices'][0]['message']['content']
                return content, used_model
            except json.JSONDecodeError as e:
                print("Error decoding JSON:", e)
                print("Raw response text:", response.text)
                time.sleep(1)  # Wait for a second before retrying
        except Exception as e:
            print("Error during API request:", e)
            time.sleep(1)  # Wait for a second before retrying

# Specify the model you want to use
model_to_use = "gemma-7b"  # Replace with your desired model

# Read messages from an Excel file
input_file = os.path.abspath('/content/drive/MyDrive/Daniel hernandez/Projects/AntiSmishGPT/datasets/cleaned_smishTank_set.xlsx')
#input_file = '/content/drive/MyDrive/Daniel hernandez/Projects/AntiSmishGPT/datasets/testFile.xlsx'
df = pd.read_excel(input_file, header=None)
df.columns = ['message']

# Process each message and get legitimacy determination
determinations = []
models = []
for i, message in enumerate(df['message'], start=1):
    determination, model = check_sms_legitimacy(message, model_to_use)
    determinations.append(determination)
    models.append(model)
    print(f"Message {i}: {determination} (Model: {model})")

# Add the determinations to the DataFrame
df['determination'] = determinations
df['model'] = models

# Save the results to a new Excel file
output_file = 'updated_cleaned_smishTank_set.xlsx'
df.to_excel(output_file, index=False)

print("Legitimacy check completed. Results saved to:", output_file)

Message 1: Undetermined. The message appears to be a potential scam, but further verification is needed to confirm its authenticity. (Model: gemma-7b)
Message 2: Undetermined. The message attempts to collect payment, but lacks verifiable information or official branding. (Model: gemma-7b)
Message 3: Fake. The email contains suspicious syntax and unsolicited request for action. (Model: gemma-7b)
Message 4: Undetermined. The message does not provide enough information to definitively determine its authenticity. (Model: gemma-7b)
Message 5: Undetermined. The message contains unusual phrasing and lacks context, making it difficult to determine its authenticity. (Model: gemma-7b)
Message 6: Undetermined. The message contains informal language and a casual tone, but it lacks the typical characteristics of an official or automated message. (Model: gemma-7b)
Message 7: Fake. The message contains suspicious elements, such as unsolicited invitation, promise of profit without investment, and a su